In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import xml.etree.ElementTree as ET
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def parse_relative_time(relative_time):
    relative_time = relative_time.lower()
    now = datetime.now()

    if "segundos" in relative_time or "minutos" in relative_time:
        # Extrair o número de minutos
        minutes = int(relative_time.split()[1])
        return now - timedelta(minutes=minutes)

    elif "horas" in relative_time:
        # Extrair o número de horas
        hours = int(relative_time.split()[1])
        return now - timedelta(hours=hours)

    elif "ontem" in relative_time:
        # Ontem
        return now - timedelta(days=1)

    elif "dia" in relative_time or "dias" in relative_time:
        # Extrair o número de dias
        days = int(relative_time.split()[1])
        return now - timedelta(days=days)

    elif "semana" in relative_time or "semanas" in relative_time:
        # Extrair o número de semanas
        weeks = int(relative_time.split()[1])
        return now - timedelta(weeks=weeks)

    elif "mês" in relative_time or "meses" in relative_time:
        # Extrair o número de meses
        months = int(relative_time.split()[1])
        # Assume-se que um "mês" tem 30 dias para simplificar
        return now - timedelta(days=months * 30)

    else:
        return None


### Extrair Notícias Brazil Journal

In [ ]:
def getNewsBrazilJournal(url, tag):

  # Inicializando o DataFrame com None
  df = None

  # Fazendo a solicitação GET
  response = requests.get(url)

  # Verificando se a solicitação foi bem-sucedida
  if response.status_code == 200:
      # Obtendo o conteúdo HTML
      html = response.text

      # Parseando o HTML com BeautifulSoup
      soup = BeautifulSoup(html, "html.parser")

      # Encontrando todas as notícias
      articles = soup.find_all("article", class_="boxarticle")

      # Lista para armazenar os dados de cada notícia
      data = []

      # Iterando sobre cada notícia para extrair título, link e data
      for article in articles:
          # print(article)
          title = article.find("h2", class_="boxarticle-infos-title").text.strip()
          link = article.find("h2", class_="boxarticle-infos-title").find("a")["href"]  # Extrai a URL da notícia
          img_elem = article.find("img")  # Encontra a tag img dentro da tag a
          img_url = img_elem["src"] if img_elem else None  # Extrai a URL da imagem, se existir


          # Fazendo solicitação GET para o link da notícia
          response_article = requests.get(link)

          # Verificando se a solicitação para o link da notícia foi bem-sucedida
          if response_article.status_code == 200:
              # Obtendo o conteúdo HTML da notícia
              html_article = response_article.text

              # Parseando o HTML da notícia com BeautifulSoup
              soup_article = BeautifulSoup(html_article, "html.parser")
              # print(soup_article)

              # Encontrando a data de publicação da notícia, se existir
              post_time_elem = soup_article.find("div", class_="post-header-info")
              post_time = post_time_elem.text.strip() if post_time_elem else None


              # Adicionando os dados da notícia à lista de dados
              data.append({
                  "titulo": title,
                  "url": link,  # Agora a URL da notícia é usada aqui
                  "descricao": "",
                  "data_publicacao": post_time,
                  "img": img_url,
                  "fonte": "Brazil Journal"
              })
          else:
              print("Falha ao acessar o link da notícia:", link)

      # Criando o DataFrame com os dados das notícias
      df = pd.DataFrame(data)

  else:
      print("Falha ao acessar o site. Status code:", response.status_code)

  return df

# URL do site
urlBrazilJournal = [
        {"url": "https://braziljournal.com/tag/petroleo/", "tag": "petroleo"},
        {"url": "https://braziljournal.com/tag/setor-eletrico/", "tag": "setor-eletrico"},
        {"url": "https://braziljournal.com/tag/servicos-financeiros/", "tag": "servicos-financeiros"},
        {"url": "https://braziljournal.com/tag/varejo-e-consumo/", "tag": "varejo-e-consumo"},
        {"url": "https://braziljournal.com/tag/mineracao/", "tag": "mineracao"},
        {"url": "https://braziljournal.com/tag/siderurgia/", "tag": "siderurgia"},
        {"url": "https://braziljournal.com/tag/industria-automotiva/", "tag": "industria-automotiva"},
        {"url": "https://braziljournal.com/tag/agronegocio/", "tag": "agronegocio"},
        {"url": "https://braziljournal.com/tag/arte/", "tag": "arte"},
        {"url": "https://braziljournal.com/tag/beleza-e-cosmeticos/", "tag": "beleza-e-cosmeticos"},
        {"url": "https://braziljournal.com/tag/drogaria/", "tag": "drogaria"}
      ]

# Inicializando o DataFrame
df_bj = pd.DataFrame()

for url in urlBrazilJournal:
  df = getNewsBrazilJournal(url["url"], url["tag"])
  df_bj = pd.concat([df, df_bj])

# dropar linhas None de publicacao
df_bj = df_bj.dropna(subset=['data_publicacao'])

df_bj


Falha ao acessar o site. Status code: 404


,titulo,url,descricao,data_publicacao,img,fonte
0,Grupo Boticário cresce 30% e ganha share; B2B ...,https://braziljournal.com/grupo-boticario-cres...,,3 de abril de 2024,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
1,"No Mundo do Cabeleireiro, uma expansão adiada ...",https://braziljournal.com/no-mundo-do-cabeleir...,,6 de março de 2024,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
2,"Advent compra a Skala, a líder dos cremes para...",https://braziljournal.com/advent-compra-a-skal...,,28 de fevereiro de 2024,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
3,Boca Rosa — e agora livre. O voo solo da influ...,https://braziljournal.com/boca-rosa-e-agora-li...,,28 de dezembro de 2023,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
4,Natura&Co: The Body Shop já tem três interessados,https://braziljournal.com/natura-the-body-shop...,,20 de setembro de 2023,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
...,...,...,...,...,...,...
11,Grupo Boticário cresce 30% e ganha share; B2B ...,https://braziljournal.com/grupo-boticario-cres...,,3 de abril de 2024,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
12,"Em dólar corrente, PIB do México já é maior qu...",https://braziljournal.com/em-dolar-corrente-pi...,,5 de abril de 2024,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
13,"Na Cruzeiro do Sul, o dilema valor x liquidez",https://braziljournal.com/long-short-na-cruzei...,,2 de abril de 2024,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
14,"Goldman: IA já aumentou produtividade nos EUA,...",https://braziljournal.com/goldman-ia-ja-aument...,,2 de abril de 2024,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal


In [ ]:
# URL do site
url = "https://braziljournal.com/"

# Fazendo a solicitação GET
response = requests.get(url)

# Verificando se a solicitação foi bem-sucedida
if response.status_code == 200:
    # Obtendo o conteúdo HTML
    html = response.text

    # Parseando o HTML com BeautifulSoup
    soup = BeautifulSoup(html, "html.parser")

    # Encontrando todas as notícias
    articles = soup.find_all("article", class_="boxarticle")

    # Lista para armazenar os dados de cada notícia
    data = []

    # Iterando sobre cada notícia para extrair título, link e data
    for article in articles:
        title = article.find("h2", class_="boxarticle-infos-title").text.strip()
        link = article.find("a")["href"]
        img_elem = article.find("img")  # Encontra a tag img dentro da tag a
        img_url = img_elem["src"] if img_elem else None  # Extrai a URL da imagem, se existir
        # tag = article.find("p", class_="boxarticle-infos-tag").text.strip()

        # Fazendo solicitação GET para o link da notícia
        response_article = requests.get(link)

        # Verificando se a solicitação para o link da notícia foi bem-sucedida
        if response_article.status_code == 200:
            # Obtendo o conteúdo HTML da notícia
            html_article = response_article.text

            # Parseando o HTML da notícia com BeautifulSoup
            soup_article = BeautifulSoup(html_article, "html.parser")

            # Encontrando a data de publicação da notícia, se existir
            post_time_elem = soup_article.find("div", class_="post-header-info")
            post_time = post_time_elem.text.strip() if post_time_elem else None

            # Adicionando os dados da notícia à lista de dados
            data.append({
                "titulo": title,
                "url": link,
                "descricao": "saiba mais a sobre "+title,
                "data_publicacao": post_time,
                "img": img_url,
                "fonte": "Brazil Journal"
            })
        else:
            print("Falha ao acessar o link da notícia:", link)

    # Criando o DataFrame com os dados das notícias
    df_bj_p = pd.DataFrame(data)

else:
    print("Falha ao acessar o site. Status code:", response.status_code)

df_bj_p = df_bj_p.dropna(subset=['data_publicacao'])
df_bj_p


,titulo,url,descricao,data_publicacao,img,fonte
42,Puig – dona da Carolina Herrera e Jean Paul Ga...,https://braziljournal.com/puig-dona-da-carolin...,saiba mais a sobre Puig – dona da Carolina Her...,8 de abril de 2024,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
47,A crise dos restaurantes estrelados,https://braziljournal.com/a-crise-dos-restaura...,saiba mais a sobre A crise dos restaurantes es...,7 de abril de 2024,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
52,"Alife Nino, dono do Nino Cucina e Tatu Bola, c...",https://braziljournal.com/alife-nino-dono-do-n...,"saiba mais a sobre Alife Nino, dono do Nino Cu...",5 de abril de 2024,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
53,Trump Media: bolha ou startup?,https://braziljournal.com/trump-media-bolha-ou...,saiba mais a sobre Trump Media: bolha ou startup?,5 de abril de 2024,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal


### Extrair Notícias do Valor Econômico

In [ ]:
# Lista para armazenar os dados de todas as páginas
all_data = []

# Loop sobre cada página até a página 100
for page_number in range(1, 11):
    # URL da página atual
    url = f"https://valor.globo.com/financas/index/feed/pagina-{page_number}"

    # Fazendo a solicitação GET
    response = requests.get(url)

    # Verificando se a solicitação foi bem-sucedida
    if response.status_code == 200:
        # Obtendo o conteúdo HTML
        html = response.text

        # Parseando o HTML com BeautifulSoup
        soup = BeautifulSoup(html, "html.parser")

        # Encontrando todas as notícias
        articles = soup.find_all("div", class_="feed-post-body")

        # Iterando sobre cada notícia para extrair título, link, data e imagem
        for article in articles:
            title_elem = article.find("h2", class_="feed-post-link")
            link_elem = article.find("a")
            description_elem = article.find("p", class_="feed-post-body-resumo")
            img_elem = article.find("img")
            post_time_elem = article.find("span", class_="feed-post-datetime")

            # Definindo valores padrão para elementos ausentes
            title = title_elem.text.strip() if title_elem else "Título não disponível"
            link = link_elem["href"] if link_elem else "Link não disponível"
            description = description_elem.text.strip() if description_elem else "Descrição não disponível"
            img = img_elem["src"] if img_elem else "Imagem não disponível"
            post_time = post_time_elem.text.strip() if post_time_elem else "Data de publicação não disponível"

            # Adicionando os dados da notícia à lista de dados de todas as páginas
            all_data.append({
                "titulo": title,
                "url": link,
                "descricao": description,
                "data_publicacao": post_time,
                "img": img,
                "fonte": "Valor Econômico"
            })

        print(f"Página {page_number} processada.")
    else:
        print(f"Falha ao acessar a página {page_number}. Status code:", response.status_code)

# Criando o DataFrame com os dados de todas as páginas
df_valor = pd.DataFrame(all_data)

df_valor["data_publicacao"] = df_valor["data_publicacao"].apply(parse_relative_time)


Página 1 processada.
Página 2 processada.
Página 3 processada.
Página 4 processada.
Página 5 processada.
Página 6 processada.
Página 7 processada.
Página 8 processada.
Página 9 processada.
Página 10 processada.


### Extrair notícias Infomoney

In [ ]:
# Função para ler o XML da URL e retornar um DataFrame
def xml_to_dataframe(url):
    # Fazendo a requisição GET para obter o conteúdo da URL
    response = requests.get(url)

    # Verificando se a requisição foi bem-sucedida
    if response.status_code == 200:
        # Analisando o XML
        root = ET.fromstring(response.content)

        # Inicializando listas para armazenar os dados
        titles = []
        links = []
        descriptions = []
        pub_dates = []
        images = []  # Nova lista para armazenar os URLs das imagens

        # Expressão regular para encontrar a tag <img> e seu conteúdo
        img_regex = re.compile(r'<img[^>]+src="([^"]+)"[^>]*>')

        # Iterando sobre os itens no XML
        for item in root.findall('.//item'):
            title = item.find('title').text
            link = item.find('link').text
            description = item.find('description').text.replace('<br />', '').replace("<p>", "").replace("</p>", "")
            pub_date = item.find('pubDate').text

            # Procurando por tags <img> na descrição e extraindo o URL da imagem
            img_match = img_regex.search(description)
            image_url = img_match.group(1) if img_match else None

            # Removendo a tag <img> da descrição
            description = img_regex.sub('', description)

            # Adicionando os dados às listas
            titles.append(title)
            links.append(link)
            descriptions.append(description)
            pub_dates.append(pub_date)
            images.append(image_url)

        # Criando um dicionário com os dados
        data = {
            'titulo': titles,
            'url': links,
            'descricao': descriptions,
            'data_publicacao': pub_dates,
            'img': images,
            "fonte": "InfoMoney"  # Alterando a fonte para "InfoMoney"
        }

        # Criando o DataFrame
        df = pd.DataFrame(data)
        return df
    else:
        print("Erro ao obter o XML. Código de status:", response.status_code)
        return None

# URL do XML do InfoMoney
url = "https://www.infomoney.com.br/feed/"

# Chamando a função para criar o DataFrame
df_infomoney = xml_to_dataframe(url)

# Exibindo o DataFrame
df_infomoney


,titulo,url,descricao,data_publicacao,img,fonte
0,"Tenda (TEND3) tem alta de 47,3% nas vendas no ...",https://www.infomoney.com.br/mercados/tenda-te...,"Velocidade de vendas atingiu 31,2%, 6,4 p.p. s...","Mon, 08 Apr 2024 22:54:23 +0000",https://www.infomoney.com.br/wp-content/upload...,InfoMoney
1,BNDES divulga novo concurso público para 150 v...,https://www.infomoney.com.br/carreira/bndes-di...,Edital será publicado no segundo semestre dest...,"Mon, 08 Apr 2024 21:54:00 +0000",https://www.infomoney.com.br/wp-content/upload...,InfoMoney
2,Oi (OIBR3) adia assembleia de credores para 17...,https://www.infomoney.com.br/mercados/oi-oibr3...,Pedido foi feito por um grupo de credores inte...,"Mon, 08 Apr 2024 21:45:26 +0000",https://www.infomoney.com.br/wp-content/upload...,InfoMoney
3,Haddad: quem tem de decidir sobre dividendos é...,https://www.infomoney.com.br/politica/haddad-q...,"Segundo ministro, Lula tem tido acesso a dados...","Mon, 08 Apr 2024 21:37:15 +0000",https://www.infomoney.com.br/wp-content/upload...,InfoMoney
4,Ação da Petrobras (PETR4) avança sob expectati...,https://www.infomoney.com.br/mercados/petr4-ma...,Ações da Petrobras têm sessão de baixo volume ...,"Mon, 08 Apr 2024 21:03:45 +0000",https://www.infomoney.com.br/wp-content/upload...,InfoMoney
5,Dengue: Nísia promete ampliar doses e vê vacin...,https://www.infomoney.com.br/politica/dengue-n...,Ministra da Saúde afirma que o governo negocia...,"Mon, 08 Apr 2024 20:59:00 +0000",https://www.infomoney.com.br/wp-content/upload...,InfoMoney
6,Haddad discute hoje com Lula pontos sensíveis ...,https://www.infomoney.com.br/mercados/haddad-d...,"Segundo ministro, existem pontos da discussão ...","Mon, 08 Apr 2024 20:55:00 +0000",https://www.infomoney.com.br/wp-content/upload...,InfoMoney
7,"Dólar hoje fecha em baixa 0,67% com ajustes de...",https://www.infomoney.com.br/mercados/dolar-ho...,"Apesar da queda, dólar permanece acima dos R$ ...","Mon, 08 Apr 2024 20:42:00 +0000",https://www.infomoney.com.br/wp-content/upload...,InfoMoney
8,Ações da Dexco (DXCO3) lideram ganhos da Bolsa...,https://www.infomoney.com.br/mercados/rascunho...,Papéis da companhia lideram ganhos do Ibovespa...,"Mon, 08 Apr 2024 20:38:31 +0000",https://www.infomoney.com.br/wp-content/upload...,InfoMoney
9,"Bolsas de NY fecham com variações marginais, e...",https://www.infomoney.com.br/mercados/bolsas-d...,Crescce expectativa por uma postura mais conse...,"Mon, 08 Apr 2024 20:35:00 +0000",https://www.infomoney.com.br/wp-content/upload...,InfoMoney


### Concatenar as Fontes

In [ ]:
df = pd.concat([df_bj, df_bj_p, df_infomoney, df_valor])

def converter_data(valor):
    try:
        # Tenta converter para data diretamente
        return pd.to_datetime(valor)
    except ValueError:
        pass

    # Verifica se o valor está no formato 'X dias atrás'
    if 'dias atrás' or 'dia atrás' in valor:
        dias_atras = int(valor.split()[0])
        return datetime.now() - relativedelta(days=dias_atras)

    # Verifica se o valor está no formato 'X semanas atrás'
    if 'semanas atrás' or 'semana atrás' in valor:
        semanas_atras = int(valor.split()[0])
        return datetime.now() - relativedelta(weeks=semanas_atras)

    # Verifica se o valor está no formato 'X de mês de ano'
    try:
        return pd.to_datetime(valor, format='%d de %B de %Y')
    except ValueError:
        pass

    # Verifica se o valor está no formato 'X de mês de ano'
    try:
        return pd.to_datetime(valor, format='%d/%m/%Y %H:%M:%S')
    except ValueError:
        pass

    # Verifica se o valor está no formato 'X de mês de ano'
    try:
        return pd.to_datetime(valor, format='%a, %d %b %Y %H:%M:%S %z')
    except ValueError:
        pass

    # Se não se encaixar em nenhum dos formatos conhecidos, retorna NaN
    return pd.NaT

# Aplicando a função de conversão à coluna 'data'
df['data_publicacao'] = df['data_publicacao'].apply(converter_data)

df

,titulo,url,descricao,data_publicacao,img,fonte
0,Grupo Boticário cresce 30% e ganha share; B2B ...,https://braziljournal.com/grupo-boticario-cres...,,2024-04-06 03:26:04.002927,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
1,"No Mundo do Cabeleireiro, uma expansão adiada ...",https://braziljournal.com/no-mundo-do-cabeleir...,,2024-04-03 03:26:04.003245,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
2,"Advent compra a Skala, a líder dos cremes para...",https://braziljournal.com/advent-compra-a-skal...,,2024-03-12 03:26:04.003469,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
3,Boca Rosa — e agora livre. O voo solo da influ...,https://braziljournal.com/boca-rosa-e-agora-li...,,2024-03-12 03:26:04.003677,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
4,Natura&Co: The Body Shop já tem três interessados,https://braziljournal.com/natura-the-body-shop...,,2024-03-20 03:26:04.003967,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
...,...,...,...,...,...,...
95,Manhã no mercado: ‘Payroll’ e Petrobras estão ...,https://valor.globo.com/financas/noticia/2024/...,"Além desses pontos, a questão fiscal se mantém...",2024-04-05 03:24:55.657966,https://s2-valor.glbimg.com/tPZeKQ2kkg0z9lC2J1...,Valor Econômico
96,Agenda do dia: ‘Payroll’ nos EUA é destaque,https://valor.globo.com/financas/noticia/2024/...,Descrição não disponível,2024-04-05 03:24:55.657968,https://s2-valor.glbimg.com/2eQ6o3Ci4-UvQVi8HE...,Valor Econômico
97,Bolsas da Europa têm queda firme seguindo Wall...,https://valor.globo.com/financas/noticia/2024/...,Pressão vem também da alta dos preços do petró...,2024-04-05 03:24:55.657970,https://s2-valor.glbimg.com/bZooT_0M_GKJqWlnX-...,Valor Econômico
98,Bolsas da Ásia caem após comentários mais rígi...,https://valor.globo.com/financas/noticia/2024/...,"Mercados da China continental, Hong Kong e Tai...",2024-04-05 03:24:55.657972,https://s2-valor.glbimg.com/QatAvpnyyGb9QyMxlz...,Valor Econômico


### Calculo de Similaridade - Verificar se as noticias estão repetindo

In [ ]:
# Criando um vetorizador TF-IDF para converter o texto das notícias em vetores numéricos
vectorizer = TfidfVectorizer()

# Convertendo o texto das notícias em vetores TF-IDF
tfidf_matrix = vectorizer.fit_transform(df['titulo'])

# Calculando a similaridade de cosseno entre os vetores TF-IDF
similaridade = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Definindo um limite de similaridade para considerar duas notícias como iguais
limite_similaridade = 0.8

# Lista para armazenar os índices das notícias únicas
indices_noticias_unicas = []

# Procurando por notícias semelhantes
for i in range(len(similaridade)):
    duplicada = False
    for j in range(i+1, len(similaridade)):
        if similaridade[i,j] > limite_similaridade:
            duplicada = True
            break
    if not duplicada:
        indices_noticias_unicas.append(i)

# Selecionando apenas as notícias únicas do dataframe original
df_noticias_unicas = df.iloc[indices_noticias_unicas]

# Agora 'df_noticias_unicas' contém apenas as notícias únicas
df = df_noticias_unicas

df

,titulo,url,descricao,data_publicacao,img,fonte
1,"No Mundo do Cabeleireiro, uma expansão adiada ...",https://braziljournal.com/no-mundo-do-cabeleir...,,2024-04-03 03:26:04.003245,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
2,"Advent compra a Skala, a líder dos cremes para...",https://braziljournal.com/advent-compra-a-skal...,,2024-03-12 03:26:04.003469,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
3,Boca Rosa — e agora livre. O voo solo da influ...,https://braziljournal.com/boca-rosa-e-agora-li...,,2024-03-12 03:26:04.003677,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
4,Natura&Co: The Body Shop já tem três interessados,https://braziljournal.com/natura-the-body-shop...,,2024-03-20 03:26:04.003967,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
5,Natura&Co pode buscar comprador para a Body Shop,https://braziljournal.com/naturaco-pode-buscar...,,2024-03-12 03:26:04.004174,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
...,...,...,...,...,...,...
95,Manhã no mercado: ‘Payroll’ e Petrobras estão ...,https://valor.globo.com/financas/noticia/2024/...,"Além desses pontos, a questão fiscal se mantém...",2024-04-05 03:24:55.657966,https://s2-valor.glbimg.com/tPZeKQ2kkg0z9lC2J1...,Valor Econômico
96,Agenda do dia: ‘Payroll’ nos EUA é destaque,https://valor.globo.com/financas/noticia/2024/...,Descrição não disponível,2024-04-05 03:24:55.657968,https://s2-valor.glbimg.com/2eQ6o3Ci4-UvQVi8HE...,Valor Econômico
97,Bolsas da Europa têm queda firme seguindo Wall...,https://valor.globo.com/financas/noticia/2024/...,Pressão vem também da alta dos preços do petró...,2024-04-05 03:24:55.657970,https://s2-valor.glbimg.com/bZooT_0M_GKJqWlnX-...,Valor Econômico
98,Bolsas da Ásia caem após comentários mais rígi...,https://valor.globo.com/financas/noticia/2024/...,"Mercados da China continental, Hong Kong e Tai...",2024-04-05 03:24:55.657972,https://s2-valor.glbimg.com/QatAvpnyyGb9QyMxlz...,Valor Econômico


### Análise de Sentimento das notícias

In [ ]:
from transformers import AutoTokenizer, BertForSequenceClassification
import numpy as np
import pandas as pd

class SentimentClassifier:
    def __init__(self, model_name="lucas-leme/FinBERT-PT-BR"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = BertForSequenceClassification.from_pretrained(model_name)
        self.pred_mapper = {
            0: "positivo",
            1: "negativo",
            2: "neutro"
        }

    def classify_sentiments(self, df, text_column):
        results = []
        for text in df[text_column]:
            tokens = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
            outputs = self.model(**tokens)
            probs = np.exp(outputs.logits.cpu().detach().numpy()) / np.exp(outputs.logits.cpu().detach().numpy()).sum(axis=1, keepdims=True)
            preds = dict(zip(self.pred_mapper.values(), probs[0]))
            results.append(preds)

        # Criar DataFrame com as colunas de probabilidades
        results_df = pd.DataFrame(results)

        # Adicionar as colunas de probabilidades ao DataFrame original usando join
        df_with_probs = df.join(results_df)

        return df_with_probs

# Exemplo de uso
# Supondo que 'df' é o seu DataFrame e 'text_column' é o nome da coluna que contém o texto das notícias
classifier = SentimentClassifier()
df = classifier.classify_sentiments(df, 'titulo')
df

tokenizer_config.json:   0%|          | 0.00/559 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/438k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

,titulo,url,descricao,data_publicacao,img,fonte,positivo,negativo,neutro
0,Lynda Benglis rompeu todas as regras. Com muit...,https://braziljournal.com/lynda-benglis-rompeu...,,2024-04-02 03:26:04.006491,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal,0.066181,0.387942,0.545877
0,Os Fiagros foram de zero a R$ 30 bi em 3 anos....,https://braziljournal.com/os-fiagros-foram-de-...,,2024-03-31 03:26:04.009600,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal,0.066181,0.387942,0.545877
0,Ferrari — uma macchina de fazer dinheiro — val...,https://braziljournal.com/ferrari-uma-macchina...,,2024-03-25 03:26:04.012551,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal,0.066181,0.387942,0.545877
0,Margem da Gerdau perto das mínimas. BTG não vê...,https://braziljournal.com/margem-da-gerdau-per...,,2024-03-26 03:26:04.015200,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal,0.066181,0.387942,0.545877
0,BTG joga a toalha na Vale; rebaixa papel para ...,https://braziljournal.com/btg-joga-a-toalha-na...,,2024-04-04 03:26:04.017849,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal,0.066181,0.387942,0.545877
...,...,...,...,...,...,...,...,...,...
95,Manhã no mercado: ‘Payroll’ e Petrobras estão ...,https://valor.globo.com/financas/noticia/2024/...,"Além desses pontos, a questão fiscal se mantém...",2024-04-05 03:24:55.657966,https://s2-valor.glbimg.com/tPZeKQ2kkg0z9lC2J1...,Valor Econômico,0.073547,0.102301,0.824152
96,Agenda do dia: ‘Payroll’ nos EUA é destaque,https://valor.globo.com/financas/noticia/2024/...,Descrição não disponível,2024-04-05 03:24:55.657968,https://s2-valor.glbimg.com/2eQ6o3Ci4-UvQVi8HE...,Valor Econômico,0.025212,0.927022,0.047765
97,Bolsas da Europa têm queda firme seguindo Wall...,https://valor.globo.com/financas/noticia/2024/...,Pressão vem também da alta dos preços do petró...,2024-04-05 03:24:55.657970,https://s2-valor.glbimg.com/bZooT_0M_GKJqWlnX-...,Valor Econômico,0.049427,0.120147,0.830426
98,Bolsas da Ásia caem após comentários mais rígi...,https://valor.globo.com/financas/noticia/2024/...,"Mercados da China continental, Hong Kong e Tai...",2024-04-05 03:24:55.657972,https://s2-valor.glbimg.com/QatAvpnyyGb9QyMxlz...,Valor Econômico,0.545801,0.113576,0.340623


### Classificar a empresa

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Supondo que 'df_noticias' seja o seu dataframe contendo as notícias e os textos das notícias estejam na coluna 'titulo'
# Supondo que 'df_empresas' seja o seu dataframe contendo os nomes das empresas e os nomes estejam na coluna 'nome'
df_noticias = df['titulo']
df_empresas = [
    {"nome": "Dexco", "ID": "123"},
    {"nome": "Petrobras", "ID": "124"},
    {"nome": "Tenda", "ID": "125"},
    {"nome": "Vale", "ID": "126"}
]

# Criando um vetorizador TF-IDF para converter o texto das notícias em vetores numéricos
vectorizer = TfidfVectorizer()

# Convertendo o texto das notícias em vetores TF-IDF
tfidf_matrix_noticias = vectorizer.fit_transform(df_noticias)

# Convertendo os nomes das empresas em vetores TF-IDF
nomes_empresas = [empresa['nome'] for empresa in df_empresas]
tfidf_matrix_empresas = vectorizer.transform(nomes_empresas)

# Calculando a similaridade de cosseno entre os vetores TF-IDF das notícias e das empresas
similaridade = cosine_similarity(tfidf_matrix_noticias, tfidf_matrix_empresas)

# Definindo um limite de similaridade para considerar uma menção como uma citação de empresa
limite_similaridade = 0.20

# Lista para armazenar os índices das notícias que mencionam empresas
indices_noticias_com_citacoes = []

# Lista para armazenar as empresas mencionadas em cada notícia
empresas_mencionadas_por_noticia = []

# Procurando por notícias que mencionam empresas
for i in range(len(similaridade)):
    empresas_mencionadas = []
    for j in range(len(similaridade[i])):
        if similaridade[i,j] > limite_similaridade:
            empresas_mencionadas.append(df_empresas[j]['nome'])
    if empresas_mencionadas:
        indices_noticias_com_citacoes.append(i)
        empresas_mencionadas_por_noticia.append(empresas_mencionadas)

empresas_mencionadas_por_noticia


NameError: name 'df_noticias_com_citacoes' is not defined

In [ ]:
import pandas as pd

# Suponha que você já tenha o dataframe df_noticias_com_citacoes

# Filtrando o dataframe para excluir as linhas onde o nome da empresa está em minúsculas
df_noticias_com_citacoes = df_noticias_com_citacoes[~df_noticias_com_citacoes['Empresas Mencionadas'].apply(lambda x: any(e.islower() for e in x))]

# Mostrando o novo dataframe após a exclusão
print(df_noticias_com_citacoes)


                                             Notícia Empresas Mencionadas
0  Tenda (TEND3) tem alta de 47,3% nas vendas no ...              [Tenda]
3  Haddad: quem tem de decidir sobre dividendos é...        [C Petrobras]
4  Ação da Petrobras (PETR4) avança sob expectati...        [C Petrobras]
8  Ações da Dexco (DXCO3) lideram ganhos da Bolsa...              [Dexco]
